<a href="https://colab.research.google.com/github/snaiws/Project_5/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
#라이브러리
import requests
from urllib import parse
import pandas as pd
import time
from IPython.display import display, clear_output

In [ ]:
#api시간제한 << 자동으로 되나??


In [63]:
#버전,티어별 닉넴 뽑기
def getSummonerName(tier,page,KEY): # II구간 get
  APIURL = f"https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/II?page={page}&api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  data = [x['summonerName'] for x in data if x['inactive'] == False]
  return data

In [60]:
#get puuid
def getPuuid(summonerName,KEY): #나중에 쓸 일 있을 수도 있으니 DB로 저장
  encodedSummonerName = parse.quote(summonerName)
  APIURL = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{encodedSummonerName}?api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  return data['puuid']

In [61]:
#get MatchId
def getMatchId(puuid,KEY): #나중에 쓸 일 있을 수도 있으니 DB로 저장
  APIURL = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=30&api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  return data

In [86]:
#getMatch
def getMatch(matchId,KEY): #나중에 쓸 일 있을 수도 있으니 DB로 저장
  APIURL = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  version = data['info']['gameVersion']
  data = data['info']['participants']
  data = [[x['individualPosition'],x['championName'],x['win']] for x in data]
  return version, data

In [29]:
getPuuid('지 감',KEY)

'5WhmVKS1x5F_yuGm_aAw9FPjZOLZuCIrCF-QFPjoNyhYi7u00j9JElyqdRaVxfRFUzS4rJAZMNpRDA'

In [81]:
getMatchId('5WhmVKS1x5F_yuGm_aAw9FPjZOLZuCIrCF-QFPjoNyhYi7u00j9JElyqdRaVxfRFUzS4rJAZMNpRDA',KEY)

['KR_5543120167',
 'KR_5526056338',
 'KR_5522415527',
 'KR_5522420875',
 'KR_5521111599',
 'KR_5514447659',
 'KR_5512092257',
 'KR_5509821933',
 'KR_5509790474',
 'KR_5509631666',
 'KR_5509467087',
 'KR_5508898670',
 'KR_5499694701',
 'KR_5463235456',
 'KR_5463214063',
 'KR_5463141600',
 'KR_5463066860',
 'KR_5463081701',
 'KR_5463014358',
 'KR_5462838447',
 'KR_5462830157',
 'KR_5462514175',
 'KR_5449833146',
 'KR_5449800948',
 'KR_5449695665',
 'KR_5449599649',
 'KR_5449528253',
 'KR_5449512079',
 'KR_5449385336',
 'KR_5403948885']

In [87]:
getMatch('KR_5543120167',KEY)

{'info': {'gameCreation': 1635851972000,
  'gameDuration': 1554,
  'gameEndTimestamp': 1635853581582,
  'gameId': 5543120167,
  'gameMode': 'CLASSIC',
  'gameName': 'teambuilder-match-5543120167',
  'gameStartTimestamp': 1635852026771,
  'gameType': 'MATCHED_GAME',
  'gameVersion': '11.21.403.3002',
  'mapId': 11,
  'participants': [{'assists': 7,
    'baronKills': 0,
    'bountyLevel': 0,
    'champExperience': 13828,
    'champLevel': 15,
    'championId': 266,
    'championName': 'Aatrox',
    'championTransform': 0,
    'consumablesPurchased': 7,
    'damageDealtToBuildings': 1348,
    'damageDealtToObjectives': 4683,
    'damageDealtToTurrets': 1348,
    'damageSelfMitigated': 21076,
    'deaths': 5,
    'detectorWardsPlaced': 4,
    'doubleKills': 0,
    'dragonKills': 0,
    'firstBloodAssist': True,
    'firstBloodKill': False,
    'firstTowerAssist': False,
    'firstTowerKill': False,
    'gameEndedInEarlySurrender': False,
    'gameEndedInSurrender': False,
    'goldEarned':

In [97]:
from IPython.display import clear_output

for i in range(10):
    print("Hello World!")
    clear_output(wait=True)


Hello World!


In [ ]:
def getData(tier,version,KEY):
  df = pd.DataFrame()
  while len(df) < 50000:
  summonerList = getSummonerName('DIAMOND','1',KEY)
  summonerList = [getPuuid(x,KEY) for x in summonerList]
  matchList = []
  for i in summonerList:
    matchList = matchList + getMatchId(i,KEY)
  matchList = list(set(matchList))
  for i in matchList:
    getMatch(matchId,KEY)


In [99]:
KEY = input()
clear_output(wait=True)
True

True

In [ ]:
#최신버전 불러오기
def getRecentVersion(KEY):
  